In [55]:
import pickle
import os

In [ ]:
with os.open("method.py", "w+") as f:
    f.write()

In [49]:
import tarfile

In [50]:
import tarfile
with tarfile.open("sample.tar", "w:gz") as tar:
    for name in ["foo", "bar", "quux"]:
        tar.add(name)

FileNotFoundError: [Errno 2] No such file or directory: 'foo'

In [ ]:
!pip3 install argo-workflows

In [3]:
# !pip3 install kubernetes

In [4]:
# !pip3 install openapi3

# Create Docker Image

In [ ]:
import docker
client = docker.from_env()

In [42]:
from io import BytesIO
from docker import APIClient
dockerfile = '''
# Shared Volume
FROM docker/whalesay:latest
'''
f = BytesIO(dockerfile.encode('utf-8'))
cli = APIClient()
response = [line for line in cli.build(
    fileobj=f, rm=True, tag='localhost:32000/yourname/volume'
)]
response

[b'{"stream":"Step 1/1 : FROM docker/whalesay:latest"}\r\n',
 b'{"stream":"\\n"}\r\n',
 b'{"status":"Pulling from docker/whalesay","id":"latest"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"e190868d63f8"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"909cd34c6fd7"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"0b9bfabab7c1"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"a3ed95caeb02"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"00bf65475aba"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"c57b6bcc83e3"}\r\n',
 b'{"status":"Waiting","progressDetail":{},"id":"a3ed95caeb02"}\r\n',
 b'{"status":"Waiting","progressDetail":{},"id":"00bf65475aba"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"8978f6879e2f"}\r\n',
 b'{"status":"Pulling fs layer","progressDetail":{},"id":"8eed3712d2cf"}\r\n',
 b'{"status":"Waiting","progressDetail":{},"id":"8978f6879e2f"}\r\n',
 b'{"s

In [26]:
# cli.login(registry="localhost:32000", username=None)

{'IdentityToken': '', 'Status': 'Login Succeeded'}

In [43]:
cli.push(repository="localhost:32000/yourname/volume")

'{"status":"The push refers to repository [localhost:32000/yourname/volume]"}\r\n{"status":"Preparing","progressDetail":{},"id":"5f70bf18a086"}\r\n{"status":"Preparing","progressDetail":{},"id":"d061ee1340ec"}\r\n{"status":"Preparing","progressDetail":{},"id":"d511ed9e12e1"}\r\n{"status":"Preparing","progressDetail":{},"id":"5f70bf18a086"}\r\n{"status":"Preparing","progressDetail":{},"id":"091abc5148e4"}\r\n{"status":"Preparing","progressDetail":{},"id":"b26122d57afa"}\r\n{"status":"Preparing","progressDetail":{},"id":"5f70bf18a086"}\r\n{"status":"Preparing","progressDetail":{},"id":"37ee47034d9b"}\r\n{"status":"Preparing","progressDetail":{},"id":"528c8710fd95"}\r\n{"status":"Preparing","progressDetail":{},"id":"1154ba695078"}\r\n{"status":"Waiting","progressDetail":{},"id":"37ee47034d9b"}\r\n{"status":"Waiting","progressDetail":{},"id":"528c8710fd95"}\r\n{"status":"Waiting","progressDetail":{},"id":"1154ba695078"}\r\n{"status":"Layer already exists","progressDetail":{},"id":"5f70bf18

 # Dockerize Python Application

In [ ]:
FROM python:3
ADD my_script.py /
CMD [ "python", "./my_script.py" ]

# Argo Workflow: single task

In [44]:
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

load_kube_config()  # loads local configuration from ~/.kube/config

In [45]:
v1alpha1 = V1alpha1Api()

In [46]:
# Create a Workflow with a single task

from argo.workflows.client.models import *

container = V1Container(name="sticazzi", image="localhost:32000/yourname/volume:latest", command=["cowsay"], args=["hello world"])

metadata = V1ObjectMeta(generate_name="hello-world-")

template = V1alpha1Template(name="whalesay", container=container)

spec = V1alpha1WorkflowSpec(entrypoint="whalesay", templates=[template])

status = V1alpha1WorkflowStatus()
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=metadata, 
    spec=spec,
    status=status)

In [47]:
v1alpha1.create_namespaced_workflow('default', workflow)

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2019, 12, 16, 12, 6, 10, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'hello-world-',
              'generation': 1,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'hello-world-plshj',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '29554',
              'self_link': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/hello-world-plshj',
              'uid': 'a8d5d0d0-49c6-45d8-997d-c3bfb1884adb'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'arguments': None,
          'artifact_repository_

In [20]:
wfs = v1alpha1.list_namespaced_workflows(namespace="default")

[(x.metadata.name, x.metadata.labels['workflows.argoproj.io/phase']) for x in wfs.items]

[('hello-world-4c65l', 'Succeeded'),
 ('hello-world-62jpd', 'Succeeded'),
 ('hello-world-d7p6n', 'Running'),
 ('hello-world-f8c44', 'Succeeded'),
 ('hello-world-fn7dl', 'Succeeded'),
 ('hello-world-mx8jp', 'Succeeded'),
 ('hello-world-shc9g', 'Succeeded')]

# Task With Input Artifact

In [93]:
import uuid

from argo.workflows.client.models import *

container = V1Container(name=str(uuid.uuid4()), image="debian:9.4", command=["sh", "-c"], args=["kubectl version"])

input_artifact = V1alpha1Artifact(name="kubectl", path="/usr/local/bin/kubectl", mode=755)

inputs = V1alpha1Inputs(artifacts=[input_artifact])

template = V1alpha1Template(name="kubectl-input-artifact", container=container, inputs=inputs)

metadata = V1ObjectMeta(generate_name="arguments-artifacts-")

artifact = V1alpha1Artifact(name="kubectl", http=V1alpha1HTTPArtifact("https://storage.googleapis.com/kubernetes-release/release/v1.8.0/bin/linux/amd64/kubectl"))

arguments = V1alpha1Arguments(artifacts=[artifact])

spec = V1alpha1WorkflowSpec(entrypoint="kubectl-input-artifact", templates=[template], arguments=arguments)

status = V1alpha1WorkflowStatus()
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=metadata, 
    spec=spec,
    status=status)

v1alpha1.create_namespaced_workflow('default', workflow)

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2019, 12, 16, 15, 5, 38, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'arguments-artifacts-',
              'generation': 1,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'arguments-artifacts-dqpsk',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '53140',
              'self_link': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/arguments-artifacts-dqpsk',
              'uid': '7d61f5f0-695f-42b4-8301-85e13704c6c0'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'arguments': {'artifacts':